In [12]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


# Trying different approaches of opening the 9GB + csv file

In [13]:
import os
import time

In [14]:
#Size of the file
os.path.getsize('custom_1988_2020.csv')

4544707885

In [15]:
# Approach one: using dask to read the file
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('custom_1988_2020.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.009428977966308594 sec


In [16]:
# Approach two: using pandas to read the file
import pandas as pd
start = time.time()
pandas_df = pd.read_csv('custom_1988_2020.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  45.63984537124634 sec


We can clearly see that pandas takes far more time than dask.

Unfortunately on my device which uses windows OS, the package ray is not available.

# Create the YAML file

In [17]:
%%writefile file.yaml
file_type: csv
dataset_name: custom_1988_2020
file_name: custom_1988_2020
table_name: custom_1988_2020
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - Year_and_Month
    - exp_imp
    - hs9
    - Customs
    - Country
    - Q1
    - Q2
    - Value

Overwriting file.yaml


In [18]:
import testutility as util
config_data = util.read_config_file("file.yaml")

In [19]:
config_data

{'file_type': 'csv',
 'dataset_name': 'custom_1988_2020',
 'file_name': 'custom_1988_2020',
 'table_name': 'custom_1988_2020',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['Year_and_Month',
  'exp_imp',
  'hs9',
  'Customs',
  'Country',
  'Q1',
  'Q2',
  'Value']}

# Validation

In [20]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
import pandas as pd
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

C:\Users\HGY\AppData\Local\Temp\ipykernel_4376\73410374.py:6: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(source_file,config_data['inbound_delimiter'])


,198801,1,103,100,000000190,0,35843,34353
0,198801,1,103,100,120991000,0,1590,4154
1,198801,1,103,100,210390900,0,4500,2565
2,198801,1,103,100,220890200,0,3000,757
3,198801,1,103,100,240220000,0,26000,40668
4,198801,1,103,100,250410000,0,5,8070


In [21]:
custom_1988_2020 = df.rename(columns={"198801": "Year_and_Month", "1": "exp_imp", "103": "hs9", "100": "Customs", "000000190": "Country", "0":"Q1", "35843": "Q2", "34353":"Value"})
custom_1988_2020.head()

,Year_and_Month,exp_imp,hs9,Customs,Country,Q1,Q2,Value
0,198801,1,103,100,120991000,0,1590,4154
1,198801,1,103,100,210390900,0,4500,2565
2,198801,1,103,100,220890200,0,3000,757
3,198801,1,103,100,240220000,0,26000,40668
4,198801,1,103,100,250410000,0,5,8070


In [10]:
util.col_header_val(custom_1988_2020,config_data)

column name and column length validation passed


1

In [11]:
# Construct the gz file

In [11]:
import datetime
import csv
import gzip


# Write csv in gz format in pipe separated text file (|)
custom_1988_2020.to_csv('custom_1988_2020.csv.gz',
                         sep='|',
                         header=True,
                         index=False,
                         quoting=csv.QUOTE_ALL,
                         compression='gzip',
                         quotechar='"',
                         doublequote=True,
                         lineterminator='\n')


In [11]:
#number of files in gz format folder
import os
os.path.getsize('G:/data_glacier/wk6/custom_1988_2020.csv.gz')

1135699983

In [23]:
custom_1988_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113607321 entries, 0 to 113607320
Data columns (total 8 columns):
 #   Column          Dtype
---  ------          -----
 0   Year_and_Month  int64
 1   exp_imp         int64
 2   hs9             int64
 3   Customs         int64
 4   Country         int64
 5   Q1              int64
 6   Q2              int64
 7   Value           int64
dtypes: int64(8)
memory usage: 6.8 GB
